In [32]:
from langchain.document_loaders import UnstructuredPDFLoader
import PyPDF2
import fitz


file = "_helpers/docs/Kaplan 2020 CFA Level II Schweser Notes eBook 1-2020.pdf"

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OSError: gpt3 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.

In [31]:
reader = fitz.open(file)
# print(reader.load_page(0))
print(reader.load_page(40).get_text("text"))


personal bankruptcy. Hutchins feels that her business partner, John Smith, is mostly to
blame for her situation since “he did not invest enough money in her investment
opportunities and caused them to fail.” Hutchins reports Smith to CFA Institute claiming
Smith violated the Code and Standards relating to misconduct. Which of the following
statements is most likely correct?
A. By reporting Smith to CFA Institute, Hutchins has misused the Professional Conduct
Program, thus violating the Code and Standards, but her poor investing and
bankruptcy have not violated the Code and Standards.
B. Hutchins’s bankruptcy reflects poorly on her professional reputation and thus violates
the Code and Standards, but her reporting of Smith does not.
C. Hutchins’s poor investing and bankruptcy, as well as her reporting of Smith, are both
violations of the Standards.
2. In which of the following has the analyst least likely committed plagiarism?
A. Julie Long takes performance projections and charts from 

In [3]:
docs = reader.load()

ImportError: cannot import name 'open_filename' from 'pdfminer.utils' (/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pdfminer/utils.py)